In [1]:
import pandas as pd

## KKBOX data

### preprocessing user_item_rating file

In [2]:
# read two txt files to get the data
df1 = pd.read_csv("KKBox_data/raw/train.txt", sep="\t", header=None)
df2 = pd.read_csv("KKBox_data/raw/test.txt", sep="\t", header=None)
# concatenate two dataframes
df = pd.concat([df1, df2], axis=0)
# add value 1 to the last column
df[2] = 1
# rename the columns
df.columns = ["userID", "itemID", "weight"]
# save the dataframe to dat file
df.to_csv("KKBox_data/user_item.dat", sep="\t", index=False)

### preprocessing graph file

In [3]:
df = pd.read_csv("KKBox_data/raw/auxiliary-mapping.txt", sep="|", header=None)
# rename the columns
df.columns = ["itemID", "genreID", "singerID", "composerID", "lyricistID"]
df

,itemID,genreID,singerID,composerID,lyricistID
0,14251,25,5130,15999,11050
1,50397,22,357,10084,9009
2,40036,25,3163,15155,12809
3,33023,33,6529,10572,9400
4,46848,33,6529,5183,3523
...,...,...,...,...,...
61900,23981,97,673,5505,8360
61901,11344,40,575,11880,10642
61902,38372,79,1227,3094,2694
61903,6768,33,2764,3863,3377


In [4]:
# create a array to store the data
data = []

for index, row in df.iterrows():
    for col in df.columns[1:]:
        if type(row[col]) == str:
            for entity_id in row[col].split(","):
                data.append([row["itemID"], col, entity_id])
        else:
            data.append([row["itemID"], col, row[col]])
# create a new dataframe to store the data with columns: itemID, type, entity_id
df_kg = pd.DataFrame(columns=["itemID", "type", "entity_id"], data=data)
# format the entity_id to int
df_kg["entity_id"] = df_kg["entity_id"].astype(int)

df_kg

,itemID,type,entity_id
0,14251,genreID,25
1,14251,singerID,5130
2,14251,composerID,15999
3,14251,lyricistID,11050
4,50397,genreID,22
...,...,...,...
250888,10575,genreID,67
250889,10575,genreID,38
250890,10575,singerID,3801
250891,10575,composerID,11782


In [5]:
# check number row same entity_id but different type
df_kg_ = (
    df_kg.groupby(["entity_id"])
    .size()
    .reset_index()
    .rename(columns={0: "count"})
    .sort_values("count", ascending=False)
)
print(df_kg_)
print(df_kg[df_kg["entity_id"] == 25])
df_kg_ = df_kg[df_kg["entity_id"] == 25]
# check number number of each type in df_kg_
df_kg_.groupby(["type"]).size().reset_index().rename(columns={0: "count"}).sort_values(
    "count", ascending=False
)

       entity_id  count
25            25  36414
24            24   6111
3940        3940   2683
86            86   2671
21            21   2557
...          ...    ...
17553      17553      1
17552      17552      1
17551      17551      1
17548      17548      1
17942      17942      1

[17943 rows x 2 columns]
        itemID     type  entity_id
0        14251  genreID         25
8        40036  genreID         25
20       18392  genreID         25
24       50839  genreID         25
37       59054  genreID         25
...        ...      ...        ...
250827   16142  genreID         25
250835   35285  genreID         25
250839    3356  genreID         25
250843   18128  genreID         25
250851    4657  genreID         25

[36414 rows x 3 columns]


,type,count
1,genreID,36407
3,singerID,5
0,composerID,1
2,lyricistID,1


In [6]:
# groupby type
df_kg_ = df_kg.groupby(["type"])

# create a list to store the df type
df_list = []

# transform the entity_id unique every type. e.g from: type = itemID-actorsID, entity_id = 0,1,2; type = itemID-directorID, entity_id = 0,1,2
# to type = itemID-actorsID, entity_id = 0,1,2; type = itemID-directorID, entity_id = 3,4,5

current_start = max(df_kg["itemID"]) + 1

# loop through each type
for name, group in df_kg_:
    print("name type: ", name)
    print("length: ", len(group))
    print("number unique entity_id: ", len(group["entity_id"].unique()))
    print("current_start: ", current_start)
    print("before transform entity_id: ")
    print(group["entity_id"].unique())
    # entity_id start from current_start
    group["entity_id"] = group["entity_id"] + current_start
    print("after transform entity_id: ")
    print(group["entity_id"].unique())
    # append the group to the list
    df_list.append(group)
    # update current_start
    current_start = current_start + max(group["entity_id"]) + 1
    print("---------------")

name type:  composerID
length:  61905
number unique entity_id:  17943
current_start:  61905
before transform entity_id: 
[15999 10084 15155 ...   347  3094 11782]
after transform entity_id: 
[77904 71989 77060 ... 62252 64999 73687]
---------------
name type:  genreID
length:  65178
number unique entity_id:  106
current_start:  141753
before transform entity_id: 
[ 25  22  33  11  85 100  24  79   4  72  48  44   3   2  86  30  21  20
  84  26  50  80  51  12  54  95  36  23  19   5  97  35  38   8  74  17
  89  43  40  67  31  94  83  57  47  45 105  66  63  69  99  76  10  32
  65   7  91  42  39  96  90  92 106  27  87  16 104  15  78   6  88  98
  29  37  46  58  75 103  49  68  62  71 102  13  77  55  53  81 101  60
   9  70  28   1  14  61  64  73  52  34  18  56  93  82 107  59]
after transform entity_id: 
[141778 141775 141786 141764 141838 141853 141777 141832 141757 141825
 141801 141797 141756 141755 141839 141783 141774 141773 141837 141779
 141803 141833 141804 141765 1418

In [7]:
# concatenate the list to a dataframe
df_kg = pd.concat(df_list, axis=0)
# add itemID- to the type
df_kg["type"] = "itemID-" + df_kg["type"]
df_kg

,itemID,type,entity_id
2,14251,itemID-composerID,77904
6,50397,itemID-composerID,71989
10,40036,itemID-composerID,77060
14,33023,itemID-composerID,72477
18,46848,itemID-composerID,67088
...,...,...,...
250873,23981,itemID-singerID,585181
250877,11344,itemID-singerID,585083
250881,38372,itemID-singerID,585735
250885,6768,itemID-singerID,587272


In [8]:
df_kg.to_csv("KKBox_data/kg.txt", sep="\t", index=False, header=False)

### preprocessing item_index2entity_id file

In [9]:
# preprocessing item_index2entity_id.txt
# read the user_item.dat file
df = pd.read_csv("KKBox_data/user_item.dat", sep="\t", header=0)
# get the unique itemID
itemID = df["itemID"].unique()
# sort the itemID
itemID.sort()
# create a dataframe
df = pd.DataFrame(columns=["itemID"], data=itemID)
# duplicate the itemID to EntityID
df["EntityID"] = df["itemID"]
# save the dataframe to txt file
df.to_csv("KKBox_data/item_index2entity_id.txt", sep="\t", index=False, header=False)

## ML100K data

### preprocessing user_item_rating file

In [2]:
# read two txt files to get the data
df1 = pd.read_csv("ML100K/raw/train.txt", sep="\t", header=None)
df2 = pd.read_csv("ML100K/raw/test.txt", sep="\t", header=None)
# concatenate two dataframes
df = pd.concat([df1, df2], axis=0)
# add value 1 to the last column
df[2] = 1
# rename the columns
df.columns = ["userID", "itemID", "weight"]
# save the dataframe to dat file
df.to_csv("ML100K/user_item.dat", sep="\t", index=False)

### preprocessing graph file

In [3]:
df = pd.read_csv("ML100K/raw/auxiliary-mapping.txt", sep="|", header=None)
# rename the columns
df.columns = ["itemID", "genreID", "directorID", "actorsID"]
df

,itemID,genreID,directorID,actorsID
0,1,"0,1,2",0,"0,1,2,3"
1,2,"3,1,4",1,"4,5,6,7"
2,3,2,"2,3,4,5","8,9,10,11"
3,4,"2,5,4",6,"12,13,14,15"
4,5,"6,7,4",7,"16,17,18,19"
...,...,...,...,...
1669,1678,6,1152,"3938,3939"
1670,1679,"5,6,8",16,"3940,2458,814,3941"
1671,1680,"2,6,12",989,"882,262,383,285"
1672,1681,"17,2",92,105


In [4]:
# create a array to store the data
data = []

for index, row in df.iterrows():
    for col in df.columns[1:]:
        if type(row[col]) == str:
            for entity_id in row[col].split(","):
                data.append([row["itemID"], col, entity_id])
        else:
            data.append([row["itemID"], col, row[col]])
# create a new dataframe to store the data with columns: itemID, type, entity_id
df_kg = pd.DataFrame(columns=["itemID", "type", "entity_id"], data=data)
# format the entity_id to int
df_kg["entity_id"] = df_kg["entity_id"].astype(int)

df_kg


,itemID,type,entity_id
0,1,genreID,0
1,1,genreID,1
2,1,genreID,2
3,1,directorID,0
4,1,actorsID,0
...,...,...,...
12338,1682,directorID,1153
12339,1682,actorsID,3942
12340,1682,actorsID,3943
12341,1682,actorsID,3944


In [5]:
# check number row same entity_id but different type
df_kg_ = (
    df_kg.groupby(["entity_id"])
    .size()
    .reset_index()
    .rename(columns={0: "count"})
    .sort_values("count", ascending=False)
)
print(df_kg_)
print(df_kg[df_kg["entity_id"] == 6])
df_kg_ = df_kg[df_kg["entity_id"] == 6]
# check number number of each type in df_kg_
df_kg_.groupby(["type"]).size().reset_index().rename(columns={0: "count"}).sort_values(
    "count", ascending=False
)

      entity_id  count
6             6    982
2             2    642
8             8    366
5             5    328
3             3    269
...         ...    ...
2274       2274      1
2276       2276      1
2277       2277      1
2278       2278      1
3945       3945      1

[3946 rows x 2 columns]
       itemID        type  entity_id
14          2    actorsID          6
28          4  directorID          6
33          5     genreID          6
42          6     genreID          6
58          8     genreID          6
...       ...         ...        ...
12307    1677     genreID          6
12313    1678     genreID          6
12318    1679     genreID          6
12326    1680     genreID          6
12337    1682     genreID          6

[982 rows x 3 columns]


,type,count
2,genreID,976
1,directorID,5
0,actorsID,1


In [6]:
# groupby type
df_kg_ = df_kg.groupby(["type"])

# create a list to store the df type
df_list = []

# transform the entity_id unique every type. e.g from: type = itemID-actorsID, entity_id = 0,1,2; type = itemID-directorID, entity_id = 0,1,2
# to type = itemID-actorsID, entity_id = 0,1,2; type = itemID-directorID, entity_id = 3,4,5

current_start = max(df_kg["itemID"]) + 1

# loop through each type
for name, group in df_kg_:
    print("name type: ", name)
    print("length: ", len(group))
    print("number unique entity_id: ", len(group["entity_id"].unique()))
    print("current_start: ", current_start)
    print("before transform entity_id: ")
    print(group["entity_id"].unique())
    # entity_id start from current_start
    group["entity_id"] = group["entity_id"] + current_start
    print("after transform entity_id: ")
    print(group["entity_id"].unique())
    # append the group to the list
    df_list.append(group)
    # update current_start
    current_start = current_start + max(group["entity_id"]) + 1
    print("---------------")

name type:  actorsID
length:  6544
number unique entity_id:  3946
current_start:  1683
before transform entity_id: 
[   0    1    2 ... 3943 3944 3945]
after transform entity_id: 
[1683 1684 1685 ... 5626 5627 5628]
---------------
name type:  directorID
length:  1824
number unique entity_id:  1154
current_start:  7312
before transform entity_id: 
[   0    1    2 ... 1151 1152 1153]
after transform entity_id: 
[7312 7313 7314 ... 8463 8464 8465]
---------------
name type:  genreID
length:  3975
number unique entity_id:  26
current_start:  15778
before transform entity_id: 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
after transform entity_id: 
[15778 15779 15780 15781 15782 15783 15784 15785 15786 15787 15788 15789
 15790 15791 15792 15793 15794 15795 15796 15797 15798 15799 15800 15801
 15802 15803]
---------------


In [7]:
# concatenate the list to a dataframe
df_kg = pd.concat(df_list, axis=0)
# add itemID- to the type
df_kg["type"] = "itemID-" + df_kg["type"]
df_kg

,itemID,type,entity_id
4,1,itemID-actorsID,1683
5,1,itemID-actorsID,1684
6,1,itemID-actorsID,1685
7,1,itemID-actorsID,1686
12,2,itemID-actorsID,1687
...,...,...,...
12326,1680,itemID-genreID,15784
12327,1680,itemID-genreID,15790
12333,1681,itemID-genreID,15795
12334,1681,itemID-genreID,15780


In [8]:
df_kg.to_csv("ML100K/kg.txt", sep="\t", index=False, header=False)

### preprocessing item_index2entity_id file

In [10]:
# read the user_item.dat file
df = pd.read_csv("ML100K/user_item.dat", sep="\t", header=0)
# get the unique itemID
itemID = df["itemID"].unique()
# sort the itemID
itemID.sort()
# create a dataframe
df = pd.DataFrame(columns=["itemID"], data=itemID)
# duplicate the itemID to EntityID
df["EntityID"] = df["itemID"]
# save the dataframe to txt file
df.to_csv("ML100K/item_index2entity_id.txt", sep="\t", index=False, header=False)